# Long Short-term Memory for Sentiment Classification

This notebook uses LSTM neural network on the [IMDB sentiment classification](https://keras.io/api/datasets/imdb/) task. This is a dataset for binary sentiment classification. 25,000 highly polar movie reviews are provided for training.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence

## Dataset

### Get the data
IMDB sentiment dataset is available in keras.datasets.

In [2]:
help(imdb.load_data)

Help on function load_data in module keras.datasets.imdb:

load_data(path='imdb.npz', num_words=None, skip_top=0, maxlen=None, seed=113, start_char=1, oov_char=2, index_from=3, **kwargs)
    Loads the [IMDB dataset](https://ai.stanford.edu/~amaas/data/sentiment/).
    
    This is a dataset of 25,000 movies reviews from IMDB, labeled by sentiment
    (positive/negative). Reviews have been preprocessed, and each review is
    encoded as a list of word indexes (integers).
    For convenience, words are indexed by overall frequency in the dataset,
    so that for instance the integer "3" encodes the 3rd most frequent word in
    the data. This allows for quick filtering operations such as:
    "only consider the top 10,000 most
    common words, but eliminate the top 20 most common words".
    
    As a convention, "0" does not stand for a specific word, but instead is used
    to encode any unknown word.
    
    Args:
      path: where to cache the data (relative to `~/.keras/dataset`).

In [3]:
max_features = 20000
maxlen = 80
# maxlen: cut texts after this number of words (among top max_features most common words)
batch_size = 32

print('Loading data...')
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)
print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

Loading data...
25000 train sequences
25000 test sequences


### Data Preprocessing

Keras has already preprocessed the data

In [4]:
print('Pad sequences (samples x time)')
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)
X_train[1]

Pad sequences (samples x time)
X_train shape: (25000, 80)
X_test shape: (25000, 80)


array([ 125,   68,    2, 6853,   15,  349,  165, 4362,   98,    5,    4,
        228,    9,   43,    2, 1157,   15,  299,  120,    5,  120,  174,
         11,  220,  175,  136,   50,    9, 4373,  228, 8255,    5,    2,
        656,  245, 2350,    5,    4, 9837,  131,  152,  491,   18,    2,
         32, 7464, 1212,   14,    9,    6,  371,   78,   22,  625,   64,
       1382,    9,    8,  168,  145,   23,    4, 1690,   15,   16,    4,
       1355,    5,   28,    6,   52,  154,  462,   33,   89,   78,  285,
         16,  145,   95])

## RNN

### Build the RNN model

In [5]:
model = keras.Sequential()
# Embedding layer turns vectors of integers into dense real vectors of fixed size
model.add(layers.Embedding(max_features, 16))
model.add(layers.SimpleRNN(32, dropout=0.2, recurrent_dropout=0.2))
model.add(layers.Dense(1, activation='sigmoid')) #sigmoid --> binary/ multiple classes: softmax

optimizer = optimizers.RMSprop(learning_rate=0.001)
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

### Inspect the model

Use the `.summary` method to print a simple description of the model

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 16)          320000    
                                                                 
 simple_rnn (SimpleRNN)      (None, 32)                1568      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 321,601
Trainable params: 321,601
Non-trainable params: 0
_________________________________________________________________


### Train the model

In [7]:
EPOCHS = 32
BATCH = 64

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)

model.fit(X_train, y_train,
          batch_size=BATCH,
          epochs=EPOCHS,
          validation_split=0.2,
          verbose = 1,
          callbacks = [early_stop])

Epoch 1/32
313/313 [==============================] - 8s 20ms/step - loss: 0.7005 - accuracy: 0.5188 - val_loss: 0.6781 - val_accuracy: 0.5812
Epoch 2/32
313/313 [==============================] - 6s 19ms/step - loss: 0.6445 - accuracy: 0.6144 - val_loss: 0.5822 - val_accuracy: 0.7016
Epoch 3/32
313/313 [==============================] - 6s 20ms/step - loss: 0.4898 - accuracy: 0.7721 - val_loss: 0.4282 - val_accuracy: 0.8076
Epoch 4/32
313/313 [==============================] - 6s 20ms/step - loss: 0.4141 - accuracy: 0.8201 - val_loss: 0.4178 - val_accuracy: 0.8188
Epoch 5/32
313/313 [==============================] - 6s 20ms/step - loss: 0.3728 - accuracy: 0.8429 - val_loss: 0.6961 - val_accuracy: 0.7614
Epoch 6/32
313/313 [==============================] - 6s 19ms/step - loss: 0.3383 - accuracy: 0.8613 - val_loss: 0.3967 - val_accuracy: 0.8282
Epoch 7/32
313/313 [==============================] - 6s 19ms/step - loss: 0.3170 - accuracy: 0.8744 - val_loss: 0.4480 - val_accuracy: 0.8314

In [8]:
_, acc = model.evaluate(X_test, y_test, batch_size=64, verbose = 0)
print("Testing set accuracy: {:.2f}%".format(acc*100))

Testing set accuracy: 82.26%


## RNN using the entire sequence instead of the last output

In [9]:
model = keras.Sequential()
# Embedding layer turns vectors of integers into dense real vectors of fixed size
model.add(layers.Embedding(max_features, 16, input_length=maxlen))
model.add(layers.SimpleRNN(32, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)) #return_sequences=True,
model.add(layers.Flatten()) #difference
model.add(layers.Dense(1, activation='sigmoid'))

optimizer = optimizers.RMSprop(learning_rate=0.001)
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 80, 16)            320000    
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 80, 32)            1568      
                                                                 
 flatten (Flatten)           (None, 2560)              0         
                                                                 
 dense_1 (Dense)             (None, 1)                 2561      
                                                                 
Total params: 324,129
Trainable params: 324,129
Non-trainable params: 0
_________________________________________________________________


In [10]:
EPOCHS = 32
BATCH = 64

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)

model.fit(X_train, y_train,
          batch_size=BATCH,
          epochs=EPOCHS,
          validation_split=0.2,
          verbose = 1,
          callbacks = [early_stop])

Epoch 1/32
313/313 [==============================] - 7s 20ms/step - loss: 0.7013 - accuracy: 0.5156 - val_loss: 0.6642 - val_accuracy: 0.5926
Epoch 2/32
313/313 [==============================] - 6s 20ms/step - loss: 0.4885 - accuracy: 0.7577 - val_loss: 0.3732 - val_accuracy: 0.8396
Epoch 3/32
313/313 [==============================] - 6s 20ms/step - loss: 0.3249 - accuracy: 0.8616 - val_loss: 0.3504 - val_accuracy: 0.8440
Epoch 4/32
313/313 [==============================] - 6s 20ms/step - loss: 0.2703 - accuracy: 0.8868 - val_loss: 0.3517 - val_accuracy: 0.8488
Epoch 5/32
313/313 [==============================] - 6s 20ms/step - loss: 0.2357 - accuracy: 0.9061 - val_loss: 0.3661 - val_accuracy: 0.8482


In [11]:
_, acc = model.evaluate(X_test, y_test, batch_size=64, verbose = 0)
print("Testing set accuracy: {:.2f}%".format(acc*100))

Testing set accuracy: 84.17%


## LSTM

### Build the model

In [12]:
model = keras.Sequential()
# Embedding layer turns vectors of integers into dense real vectors of fixed size
model.add(layers.Embedding(max_features, 16))
model.add(layers.LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(layers.Dense(1, activation='sigmoid'))

optimizer = optimizers.RMSprop(learning_rate=0.001)
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

### Inspect the model

Use the `.summary` method to print a simple description of the model

In [13]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 16)          320000    
                                                                 
 lstm (LSTM)                 (None, 128)               74240     
                                                                 
 dense_2 (Dense)             (None, 1)                 129       
                                                                 
Total params: 394,369
Trainable params: 394,369
Non-trainable params: 0
_________________________________________________________________


### Train the model

In [14]:
EPOCHS = 32
BATCH = 64

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)

model.fit(X_train, y_train,
          batch_size=BATCH,
          epochs=EPOCHS,
          validation_split=0.2,
          verbose = 1,
          callbacks = [early_stop])

Epoch 1/32
313/313 [==============================] - 32s 94ms/step - loss: 0.5304 - accuracy: 0.7358 - val_loss: 0.3666 - val_accuracy: 0.8378
Epoch 2/32
313/313 [==============================] - 30s 96ms/step - loss: 0.3391 - accuracy: 0.8613 - val_loss: 0.3681 - val_accuracy: 0.8434
Epoch 3/32
313/313 [==============================] - 29s 93ms/step - loss: 0.2857 - accuracy: 0.8853 - val_loss: 0.3620 - val_accuracy: 0.8440
Epoch 4/32
313/313 [==============================] - 35s 111ms/step - loss: 0.2520 - accuracy: 0.9004 - val_loss: 0.3544 - val_accuracy: 0.8434
Epoch 5/32
313/313 [==============================] - 42s 134ms/step - loss: 0.2307 - accuracy: 0.9125 - val_loss: 0.4855 - val_accuracy: 0.8324
Epoch 6/32
313/313 [==============================] - 40s 129ms/step - loss: 0.2124 - accuracy: 0.9194 - val_loss: 0.3714 - val_accuracy: 0.8416


In [15]:
_, acc = model.evaluate(X_test, y_test, batch_size=64, verbose = 0)
print("Testing set accuracy: {:.2f}%".format(acc*100))

Testing set accuracy: 83.10%


## Stacked LSTM

### Build the model

In [16]:
model = keras.Sequential()
# Embedding layer turns vectors of integers into dense real vectors of fixed size
model.add(layers.Embedding(max_features, 16))
model.add(layers.LSTM(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2))
model.add(layers.LSTM(128, return_sequences=False, dropout=0.2, recurrent_dropout=0.2))
model.add(layers.Dense(1, activation='sigmoid'))

optimizer = optimizers.RMSprop(learning_rate=0.001)
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, None, 16)          320000    
                                                                 
 lstm_1 (LSTM)               (None, None, 128)         74240     
                                                                 
 lstm_2 (LSTM)               (None, 128)               131584    
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 525,953
Trainable params: 525,953
Non-trainable params: 0
_________________________________________________________________


### Train the model

In [17]:
EPOCHS = 32
BATCH = 64

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)

model.fit(X_train, y_train,
          batch_size=BATCH,
          epochs=EPOCHS,
          validation_split=0.2,
          verbose = 1,
          callbacks = [early_stop])

Epoch 1/32
313/313 [==============================] - 90s 275ms/step - loss: 0.5127 - accuracy: 0.7369 - val_loss: 0.4166 - val_accuracy: 0.8228
Epoch 2/32
313/313 [==============================] - 108s 345ms/step - loss: 0.3287 - accuracy: 0.8629 - val_loss: 0.3897 - val_accuracy: 0.8268
Epoch 3/32
313/313 [==============================] - 104s 333ms/step - loss: 0.2790 - accuracy: 0.8866 - val_loss: 0.3626 - val_accuracy: 0.8418
Epoch 4/32
313/313 [==============================] - 110s 353ms/step - loss: 0.2492 - accuracy: 0.9020 - val_loss: 0.3612 - val_accuracy: 0.8464
Epoch 5/32
313/313 [==============================] - 110s 353ms/step - loss: 0.2256 - accuracy: 0.9139 - val_loss: 0.4048 - val_accuracy: 0.8368
Epoch 6/32
313/313 [==============================] - 111s 354ms/step - loss: 0.2119 - accuracy: 0.9196 - val_loss: 0.4254 - val_accuracy: 0.8426


In [18]:
_, acc = model.evaluate(X_test, y_test, batch_size=64, verbose = 0)
print("Testing set accuracy: {:.2f}%".format(acc*100))

Testing set accuracy: 83.53%


## Bidirectional LSTM

In [19]:
model = keras.Sequential()
# Embedding layer turns vectors of integers into dense real vectors of fixed size
model.add(layers.Embedding(max_features, 16))
model.add(layers.Bidirectional(layers.LSTM(128, return_sequences=False, dropout=0.2, recurrent_dropout=0.2)))
model.add(layers.Dense(1, activation='sigmoid'))

optimizer = optimizers.RMSprop(learning_rate=0.001)
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, None, 16)          320000    
                                                                 
 bidirectional (Bidirectiona  (None, 256)              148480    
 l)                                                              
                                                                 
 dense_4 (Dense)             (None, 1)                 257       
                                                                 
Total params: 468,737
Trainable params: 468,737
Non-trainable params: 0
_________________________________________________________________


### Train the model

In [20]:
EPOCHS = 32
BATCH = 64

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)

model.fit(X_train, y_train,
          batch_size=BATCH,
          epochs=EPOCHS,
          validation_split=0.2,
          verbose = 1,
          callbacks = [early_stop])

_, acc = model.evaluate(X_test, y_test, batch_size=64, verbose = 0)
print("Testing set accuracy: {:.2f}%".format(acc*100))

Epoch 1/32
313/313 [==============================] - 74s 222ms/step - loss: 0.5246 - accuracy: 0.7365 - val_loss: 0.3829 - val_accuracy: 0.8352
Epoch 2/32
313/313 [==============================] - 87s 279ms/step - loss: 0.3359 - accuracy: 0.8602 - val_loss: 0.3953 - val_accuracy: 0.8412
Epoch 3/32
313/313 [==============================] - 90s 289ms/step - loss: 0.2743 - accuracy: 0.8908 - val_loss: 0.4490 - val_accuracy: 0.8216
Testing set accuracy: 82.28%


In [21]:
_, acc = model.evaluate(X_test, y_test, batch_size=64, verbose = 0)
print("Testing set accuracy: {:.2f}%".format(acc*100))

Testing set accuracy: 82.28%
